### 모듈

In [116]:
import numpy as np
import pandas as pd
import ast
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

### NBC 모델링

In [117]:
df = pd.read_csv('labeled_economy.csv', parse_dates=['date'])

In [118]:
def parse_ngram_list(x):
    return ast.literal_eval(x)

df['ngram_list'] = df['ngram_label'].apply(parse_ngram_list)

In [119]:
def create_document_and_target(ngram_list):
    if not ngram_list:
        return "", None
    
    # 초기화
    tokens = []
    target = None

    for item in ngram_list:
        token_tuple, freq, lab = item
        # item이 (('금리',), 13, 0) 형태일 때,
        # item[0] = 토큰 튜플
        # item[1] = 빈도수(freq)
        # item[2] = 라벨(0이면 dovish, 1이면 hawkish)

        # 튜플을 문자열로 만들어 합친다.
        token_str = "_".join(token_tuple)
        if token_str not in tokens:
            tokens.append(token_str)

        if lab == 1:
            target = "hawkish"
        elif lab == -1:
            target = "dovish"

    document = " ".join(tokens) if tokens else ""
    return document, target

In [120]:
df[['document', 'target']] = df['ngram_list'].apply(lambda x: pd.Series(create_document_and_target(x)))
df = df.dropna(subset=['document', 'target'])

### 학습

In [121]:
# 피처(X)와 타겟(y) 준비
X = df['document']
y = df['target']

# 학습/테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# CountVectorizer로 문서 벡터화
vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# 나이브 베이즈 분류기(MultinomialNB) 학습
clf = MultinomialNB()
clf.fit(X_train_vec, y_train)

# 테스트 데이터 예측 및 평가
y_pred = clf.predict(X_test_vec)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

      dovish       0.64      1.00      0.78        16
     hawkish       1.00      0.50      0.67        18

    accuracy                           0.74        34
   macro avg       0.82      0.75      0.72        34
weighted avg       0.83      0.74      0.72        34



In [122]:
feature_names = vectorizer.get_feature_names_out()
probs = np.exp(clf.feature_log_prob_)
print("클래스 순서:", clf.classes_)

클래스 순서: ['dovish' 'hawkish']


In [123]:
sentiment_score = probs[1] - probs[0]

sentiment_df = pd.DataFrame({
    'feature': feature_names,
    'dovish_prob': probs[0],
    'hawkish_prob': probs[1],
    'sentiment_score': sentiment_score
})

print("Hawkish 쪽 상위 단어:")
print(sentiment_df.sort_values('sentiment_score', ascending=False).head(10))
print("\nDovish 쪽 상위 단어:")
print(sentiment_df.sort_values('sentiment_score', ascending=True).head(10))

Hawkish 쪽 상위 단어:
     feature  dovish_prob  hawkish_prob  sentiment_score
1948      인상     0.002567      0.006239         0.003673
498    금리_인상     0.001497      0.004771         0.003274
915       물가     0.004919      0.007707         0.002788
703       대비     0.004385      0.007065         0.002680
26       fed     0.004919      0.007432         0.002513
1423     소비자     0.002887      0.005230         0.002342
1969   인플레이션     0.001818      0.004129         0.002311
612       긴축     0.000321      0.002569         0.002248
2206      전치     0.000107      0.002110         0.002003
1643      압력     0.001497      0.003395         0.001898

Dovish 쪽 상위 단어:
     feature  dovish_prob  hawkish_prob  sentiment_score
2394      중국     0.010908      0.006606        -0.004302
1975      인하     0.004278      0.001285        -0.002993
507    금리_인하     0.003529      0.000734        -0.002795
236       경제     0.008555      0.005964        -0.002591
937       미국     0.012512      0.010184        -0.0023

In [124]:
# target 별로 모든 document를 하나의 텍스트로 합치기
grouped_docs = df.groupby('target')['document'].apply(lambda docs: " ".join(docs)).reset_index()

In [125]:
for idx, row in grouped_docs.iterrows():
    print(f"{row['target']} : {row['document'][:200]}...")

dovish : 한국 미국 제조업 경기 수출 불확실성 상승 지수 SP 생산 감소 추세 지속 정부 기업 정책 기대 내년 고용 서비스업 임금 상승률 둔화 확대 발표 ISM PMI 흐름 필요 국내 트럼프 수입 중국 증가 크 우려 PCE 확인 소비 연말 판단 가운데 매크로 이벤트 비농업 신규 실업률 주요 유로존 기준 하락 속도 경제 독일 금리 인하 관련 인플레이션 압력 서비스 소...
hawkish : 내용 작성 확인 투자 판단 증권 물가 상승 미국 인플레이션 높 상승률 평균 경제 수요 한국 유지 지속 발표 국가 이벤트 대비 분기 전월 에너지 제외 비 수입 지수 소매 판매 가스 생산 주택 매크로 소매_판매 고용 소비자 fed 시장 가격 가중치 소비 데이터 기준 비중 서비스 하락 증가 주거비 크 주요 금리 인상 둔화 경기 유로존 소비자_물가 금리_인상 팀 ...
